In [0]:
# dbutils.widgets.text("entity","","Entity")
# entity = dbutils.widgets.get("entity")
# print(entity)
entity = "conversations"

In [0]:
from pyspark.sql.functions import *

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS slack_etl.bronze.metadata

In [0]:
bronze_df = (
    spark.readStream.format('cloudFiles')
    .option('cloudFiles.format','json')
    .option('cloudFiles.schemaLocation',f'/Volumes/slack_etl/bronze/metadata/schemas/{entity}')
    .option('cloudFiles.schemaEvolutionMode','addNewColumns')
    .option('cloudFiles.rescuedDataColumn','_rescued_data')
    .option('cloudFiles.inferColumnTypes','true')
    .option('cloudFiles.maxFilesPerTrigger','1')
    .load(f'/Volumes/slack_etl/landing/raw/{entity}/')
    )


In [0]:
bronze_df = bronze_df.withColumns({
    'ingest_ts' : current_timestamp(),
    'source_file' :col("_metadata.file_path")
})


In [0]:
bronze_df.writeStream.format('delta').option('checkpointLocation',f'/Volumes/slack_etl/bronze/metadata/checkpoints/{entity}/').outputMode('append').trigger(availableNow=True).toTable(f'slack_etl.bronze.{entity}')